In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import plotly.express as px
import datetime
import statsmodels.api as sm

In [4]:
df_indicator = pd.read_csv("data/nchs_covid_indicators_of_anxiety_depression.csv")
df_indicator["Time Period End Date"] = df_indicator["Time Period End Date"].apply(pd.to_datetime)

In [6]:
df_indicator.head()

,Indicator,Group,State,Subgroup,Phase,Time Period,Time Period Label,Time Period Start Date,Time Period End Date,Value,Low CI,High CI,Confidence Interval,Quartile Range
0,Symptoms of Depressive Disorder,National Estimate,United States,United States,1,1,"Apr 23 - May 5, 2020",04/23/2020,2020-05-05,23.5,22.7,24.3,22.7 - 24.3,NaN
1,Symptoms of Depressive Disorder,By Age,United States,18 - 29 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,2020-05-05,32.7,30.2,35.2,30.2 - 35.2,NaN
2,Symptoms of Depressive Disorder,By Age,United States,30 - 39 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,2020-05-05,25.7,24.1,27.3,24.1 - 27.3,NaN
3,Symptoms of Depressive Disorder,By Age,United States,40 - 49 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,2020-05-05,24.8,23.3,26.2,23.3 - 26.2,NaN
4,Symptoms of Depressive Disorder,By Age,United States,50 - 59 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,2020-05-05,23.2,21.5,25.0,21.5 - 25.0,NaN


In [52]:
df_indicator["Indicator"].value_counts()

Symptoms of Depressive Disorder                        3228
Symptoms of Anxiety Disorder or Depressive Disorder    3228
Symptoms of Anxiety Disorder                           3228
Name: Indicator, dtype: int64

In [39]:
df_indicator["KEY"] = (df_indicator["Group"] +
                       ", "+df_indicator["State"] +
                       ", " + df_indicator["Subgroup"] +
                       ", " + df_indicator["Time Period Label"]
                       )

In [40]:
df_indicator["KEY"].value_counts()

By State, Kentucky, Kentucky, June 25 - June 30, 2020                                       3
By State, Virginia, Virginia, May 7 - May 12, 2020                                          3
By State, Oregon, Oregon, May 12 - May 24, 2021                                             3
By State, Massachusetts, Massachusetts, Aug 19 - Aug 31, 2020                               3
By State, Virginia, Virginia, May 12 - May 24, 2021                                         3
                                                                                           ..
By State, Maryland, Maryland, May 28 - June 2, 2020                                         3
By Education, United States, Less than a high school diploma, Dec 22, 2020 - Jan 5, 2021    3
By State, Nevada, Nevada, June 11 - June 16, 2020                                           3
By Age, United States, 50 - 59 years, Dec 9 - Dec 21, 2020                                  3
By State, Connecticut, Connecticut, May 12 - May 24, 2021   

In [41]:
df_access = pd.read_csv("data/nchs_covid_mental_health_care.csv")
df_access["Time Period End Date"] = df_access["Time Period End Date"].apply(pd.to_datetime)

In [42]:
df_access.head()

,Indicator,Group,State,Subgroup,Phase,Time Period,Time Period Label,Time Period Start Date,Time Period End Date,Value,LowCI,HighCI,Confidence Interval,Quartile Range,Suppression Flag
0,Took Prescription Medication for Mental Health...,National Estimate,United States,United States,2,13,"Aug 19 - Aug 31, 2020",08/19/2020,2020-08-31,19.4,19.0,19.8,19.0 - 19.8,NaN,NaN
1,Took Prescription Medication for Mental Health...,By Age,United States,18 - 29 years,2,13,"Aug 19 - Aug 31, 2020",08/19/2020,2020-08-31,18.7,17.2,20.3,17.2 - 20.3,NaN,NaN
2,Took Prescription Medication for Mental Health...,By Age,United States,30 - 39 years,2,13,"Aug 19 - Aug 31, 2020",08/19/2020,2020-08-31,18.3,17.3,19.2,17.3 - 19.2,NaN,NaN
3,Took Prescription Medication for Mental Health...,By Age,United States,40 - 49 years,2,13,"Aug 19 - Aug 31, 2020",08/19/2020,2020-08-31,20.4,19.5,21.3,19.5 - 21.3,NaN,NaN
4,Took Prescription Medication for Mental Health...,By Age,United States,50 - 59 years,2,13,"Aug 19 - Aug 31, 2020",08/19/2020,2020-08-31,21.2,20.2,22.2,20.2 - 22.2,NaN,NaN


In [53]:
df_access["Indicator"].value_counts()

Took Prescription Medication for Mental Health And/Or Received Counseling or Therapy, Last 4 Weeks    2441
Took Prescription Medication for Mental Health, Last 4 Weeks                                          2441
Needed Counseling or Therapy But Did Not Get It, Last 4 Weeks                                         2441
Received Counseling or Therapy, Last 4 Weeks                                                          2441
Name: Indicator, dtype: int64

In [43]:
df_access["KEY"] = (df_access["Group"] +
                    ", "+df_access["State"] +
                    ", " + df_access["Subgroup"] +
                    ", " + df_access["Time Period Label"]
                    )

In [44]:
df_access["KEY"].value_counts()

By Age, United States, 50 - 59 years, Aug 4 - Aug 16, 2021         4
By State, Hawaii, Hawaii, Jan 26 - Feb 7, 2022                     4
By Age, United States, 60 - 69 years, Aug 4 - Aug 16, 2021         4
By State, New York, New York, Sep 29 - Oct 11, 2021                4
By State, West Virginia, West Virginia, Jan 6 - Jan 18, 2021       4
                                                                  ..
By State, Ohio, Ohio, Mar 3 - Mar 15, 2021                         4
By State, North Carolina, North Carolina, Apr 14 - Apr 26, 2021    4
By State, South Dakota, South Dakota, Mar 17 - Mar 29, 2021        4
By State, South Dakota, South Dakota, Aug 18 - Aug 30, 2021        4
By State, Connecticut, Connecticut, May 12 - May 24, 2021          4
Name: KEY, Length: 2441, dtype: int64

In [49]:
df_merged = pd.merge(df_access, df_indicator, on="KEY", how="left")

In [50]:
df_merged["KEY"].value_counts()

By Age, United States, 50 - 59 years, Aug 4 - Aug 16, 2021                                                                                                    12
By Age, United States, 18 - 29 years, Dec 9 - Dec 21, 2020                                                                                                    12
By State, Florida, Florida, May 12 - May 24, 2021                                                                                                             12
By Age, United States, 70 - 79 years, May 12 - May 24, 2021                                                                                                   12
By Sexual orientation, United States, Bisexual, Feb 23 - Mar 1, 2022                                                                                          12
                                                                                                                                                              ..
By Presence of Symptoms of Anxiety

In [51]:
df_merged.query("KEY == 'By State, Connecticut, Connecticut, May 12 - May 24, 2021'")

,Indicator_x,Group_x,State_x,Subgroup_x,Phase_x,Time Period_x,Time Period Label_x,Time Period Start Date_x,Time Period End Date_x,Value_x,...,Phase_y,Time Period_y,Time Period Label_y,Time Period Start Date_y,Time Period End Date_y,Value_y,Low CI,High CI,Confidence Interval_y,Quartile Range_y
15019,Took Prescription Medication for Mental Health...,By State,Connecticut,Connecticut,3.1,30,"May 12 - May 24, 2021",05/12/2021,2021-05-24,23.2,...,3.1,30.0,"May 12 - May 24, 2021",05/12/2021,2021-05-24,22.4,18.8,26.3,18.8 - 26.3,20.7-22.7
15020,Took Prescription Medication for Mental Health...,By State,Connecticut,Connecticut,3.1,30,"May 12 - May 24, 2021",05/12/2021,2021-05-24,23.2,...,3.1,30.0,"May 12 - May 24, 2021",05/12/2021,2021-05-24,27.4,23.0,32.2,23.0 - 32.2,27.1-29.8
15021,Took Prescription Medication for Mental Health...,By State,Connecticut,Connecticut,3.1,30,"May 12 - May 24, 2021",05/12/2021,2021-05-24,23.2,...,3.1,30.0,"May 12 - May 24, 2021",05/12/2021,2021-05-24,32.4,28.0,37.1,28.0 - 37.1,32.4-33.7
15237,"Received Counseling or Therapy, Last 4 Weeks",By State,Connecticut,Connecticut,3.1,30,"May 12 - May 24, 2021",05/12/2021,2021-05-24,14.9,...,3.1,30.0,"May 12 - May 24, 2021",05/12/2021,2021-05-24,22.4,18.8,26.3,18.8 - 26.3,20.7-22.7
15238,"Received Counseling or Therapy, Last 4 Weeks",By State,Connecticut,Connecticut,3.1,30,"May 12 - May 24, 2021",05/12/2021,2021-05-24,14.9,...,3.1,30.0,"May 12 - May 24, 2021",05/12/2021,2021-05-24,27.4,23.0,32.2,23.0 - 32.2,27.1-29.8
15239,"Received Counseling or Therapy, Last 4 Weeks",By State,Connecticut,Connecticut,3.1,30,"May 12 - May 24, 2021",05/12/2021,2021-05-24,14.9,...,3.1,30.0,"May 12 - May 24, 2021",05/12/2021,2021-05-24,32.4,28.0,37.1,28.0 - 37.1,32.4-33.7
15455,Took Prescription Medication for Mental Health...,By State,Connecticut,Connecticut,3.1,30,"May 12 - May 24, 2021",05/12/2021,2021-05-24,29.4,...,3.1,30.0,"May 12 - May 24, 2021",05/12/2021,2021-05-24,22.4,18.8,26.3,18.8 - 26.3,20.7-22.7
15456,Took Prescription Medication for Mental Health...,By State,Connecticut,Connecticut,3.1,30,"May 12 - May 24, 2021",05/12/2021,2021-05-24,29.4,...,3.1,30.0,"May 12 - May 24, 2021",05/12/2021,2021-05-24,27.4,23.0,32.2,23.0 - 32.2,27.1-29.8
15457,Took Prescription Medication for Mental Health...,By State,Connecticut,Connecticut,3.1,30,"May 12 - May 24, 2021",05/12/2021,2021-05-24,29.4,...,3.1,30.0,"May 12 - May 24, 2021",05/12/2021,2021-05-24,32.4,28.0,37.1,28.0 - 37.1,32.4-33.7
15673,Needed Counseling or Therapy But Did Not Get I...,By State,Connecticut,Connecticut,3.1,30,"May 12 - May 24, 2021",05/12/2021,2021-05-24,9.4,...,3.1,30.0,"May 12 - May 24, 2021",05/12/2021,2021-05-24,22.4,18.8,26.3,18.8 - 26.3,20.7-22.7
